In [30]:
# ライブラリをインポート

# データ操作と数値計算のため
import pandas as pd
import numpy as np

# データ可視化のため
import matplotlib.pyplot as plt
import seaborn as sns

# 統計モデリングと計量経済分析のため
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from scipy import stats

# for ignore warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = "https://www.fbc.keio.ac.jp/~tyabu/keiryo/dailyint_data.csv"
df = pd.read_csv(url)
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')
df = df.dropna()
df.head()

,date,spot,intervention
0,1991-04-01,139.30,0.0
1,1991-04-02,137.90,0.0
2,1991-04-03,137.38,0.0
3,1991-04-04,135.95,0.0
4,1991-04-05,136.60,0.0


In [3]:
df.describe()

,date,spot,intervention
count,3055,3055.000000,3055.000000
mean,1997-02-10 05:24:45.957446912,116.043676,-76.739771
min,1991-04-01 00:00:00,80.620000,-14059.000000
25%,1994-03-05 12:00:00,106.440000,0.000000
50%,1997-02-10 00:00:00,116.900000,0.000000
75%,2000-01-18 12:00:00,124.695000,0.000000
max,2002-12-31 00:00:00,147.200000,26201.000000
std,NaN,12.711133,883.624841


# 変数の定義
ドル円レートは変化率に、介入額(単位億円)は介入額(単位兆円)に変換

In [4]:
df['ds'] = 100 * ((np.log(df['spot'])) - np.log(df['spot'].shift()))
df['intj'] = df['intervention'] / 10000
df = df.dropna()
df.head()

,date,spot,intervention,ds,intj
1,1991-04-02,137.90,0.0,-1.010110,0.0
2,1991-04-03,137.38,0.0,-0.377798,0.0
3,1991-04-04,135.95,0.0,-1.046364,0.0
4,1991-04-05,136.60,0.0,0.476978,0.0
5,1991-04-08,137.10,0.0,0.365364,0.0


# 7.4.2節の推定結果
supF検定と構造変化日の特定

In [72]:
endog = df['ds']
exog = df['intj']
exog = sm.add_constant(exog)

model_1 = sm.OLS(endog, exog)
results_1 = model_1.fit()
print(results_1.summary())

                            OLS Regression Results                            
Dep. Variable:                     ds   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     47.67
Date:                Mon, 23 Sep 2024   Prob (F-statistic):           6.11e-12
Time:                        19:26:51   Log-Likelihood:                -3316.9
No. Observations:                3053   AIC:                             6638.
Df Residuals:                    3051   BIC:                             6650.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0127      0.013     -0.974      0.3

In [85]:
# 構造変化の検出関数
def wald_test(endog, exog, q, breakpoint):
    n = len(endog)
    k = exog.shape[1]

    exog1 = exog[:breakpoint]
    exog2 = exog[breakpoint:]
    endog1 = endog[:breakpoint]
    endog2 = endog[breakpoint:]

    model = sm.OLS(endog, exog).fit()
    model1 = sm.OLS(endog1, exog1).fit()
    model2 = sm.OLS(endog2, exog2).fit()

    rss = model.ssr
    rss1 = model1.ssr
    rss2 = model2.ssr

    f_stat = ((rss - (rss1 + rss2)) / k) / ((rss1 + rss2) / (n - 2 * k))
    wald_stat = f_stat * q
    f_p_value = 1 - stats.f.cdf(f_stat, k, n - 2*k)
    wald_p_value = 1 - stats.f.cdf(wald_stat, k, n - 2*k)

    return f_stat, wald_stat, f_p_value, wald_p_value

In [82]:
T = len(df)
T_B_min = round(T * 0.15)
T_B_max = round(T * (1-0.15))
print(T_B_min, T_B_max)

458 2595


In [86]:
# すべての可能な分割点でWaldテストを実行
f_statistics = []
wald_statistics = []
p_values = []
T = len(df)
T_B_min = round(T * 0.15)
T_B_max = round(T * (1-0.15))
# endog = df['ds']
# exog = sm.add_constant(df['intj'])
q = 2

for i in range(T_B_min, T_B_max+1):  # データの両端15%をトリミング
    f_stat, wald_stat, f_p_val, wald_p_val = wald_test(endog, exog, q, i)
    f_statistics.append(f_stat)
    wald_statistics.append(wald_stat)
    p_values.append(f_p_val)

# 最大のWald統計量を持つ点を構造変化点として特定
breakpoint = np.argmax(f_statistics) + T_B_min
# breakpoint = np.argmax(wald_statistics) + T_B_min
breakpoint_date = df.iloc[breakpoint-1]['date']
max_f_stat = np.max(f_statistics)
# max_wald_stat = np.max(wald_statistics)
min_p_value = np.min(p_values)

print(f"Estimated break point: {breakpoint}")
print(f"Estimated break point date: {breakpoint_date}")
print(f"Maximum Wald statistic: {max_f_stat:.4f}")
print(f"Minimum p-value: {min_p_value:.4f}")

Estimated break point: 1054
Estimated break point date: 1995-04-18 00:00:00
Maximum Wald statistic: 16.9219
Minimum p-value: 0.0000


前半(1995/4/18まで)と後半(1995/4/19から)で分けて、別々に推定する

In [16]:
subset_condition_1 = (df['date'] <= '1995-04-18')
Out2 = sm.OLS(df.loc[subset_condition_1, 'ds'], sm.add_constant(df.loc[subset_condition_1, 'intj'])).fit()
print(Out2.summary())

                            OLS Regression Results                            
Dep. Variable:                     ds   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     20.89
Date:                Sun, 22 Sep 2024   Prob (F-statistic):           5.43e-06
Time:                        00:15:34   Log-Likelihood:                -1020.4
No. Observations:                1055   AIC:                             2045.
Df Residuals:                    1053   BIC:                             2055.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0301      0.020     -1.493      0.1

In [17]:
subset_condition_2 = (df['date'] > '1995-04-18')
Out3 = sm.OLS(df.loc[subset_condition_2, 'ds'], sm.add_constant(df.loc[subset_condition_2, 'intj'])).fit()
print(Out3.summary())

                            OLS Regression Results                            
Dep. Variable:                     ds   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     52.90
Date:                Sun, 22 Sep 2024   Prob (F-statistic):           5.02e-13
Time:                        00:15:41   Log-Likelihood:                -2263.1
No. Observations:                1999   AIC:                             4530.
Df Residuals:                    1997   BIC:                             4541.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0095      0.017      0.566      0.5

推定結果の頑健性を調べるため、説明変数に新たにdsの1期前の値を加える

In [54]:
# Adding lagged values of 'ds' to the dataframe
df['lag_ds'] = df['ds'].shift()

# Dropping NaN values that result from the lag operation
df = df.dropna()

# Fitting the model
exog = df[['intj', 'lag_ds']]
exog = sm.add_constant(exog)
endog = df['ds']
Out4 = sm.OLS(endog, exog).fit()

# Summarizing the model
print(Out4.summary())

                            OLS Regression Results                            
Dep. Variable:                     ds   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     24.27
Date:                Mon, 23 Sep 2024   Prob (F-statistic):           3.50e-11
Time:                        19:10:24   Log-Likelihood:                -3316.5
No. Observations:                3053   AIC:                             6639.
Df Residuals:                    3050   BIC:                             6657.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0127      0.013     -0.976      0.3

In [59]:
# すべての可能な分割点でWaldテストを実行
wald_statistics = []
p_values = []
T = len(df)
T_B_min = round(T * 0.15)
T_B_max = round(T * (1-0.15))
endog = df['ds']
exog = sm.add_constant(df[['intj', 'lag_ds']])
q = 3

for i in range(T_B_min, T_B_max+1):  # データの両端15%をトリミング
    chow_stat, p_val = wald_test(endog, exog, q, i)
    wald_statistics.append(chow_stat)
    p_values.append(p_val)

# 最大のWald統計量を持つ点を構造変化点として特定
breakpoint = np.argmax(wald_statistics) + T_B_min
breakpoint_date = df.iloc[breakpoint-1]['date']
max_wald_stat = np.max(wald_statistics)
min_p_value = np.min(p_values)

print(f"Estimated break point date: {breakpoint_date}")
print(f"Maximum Wald statistic: {max_wald_stat:.4f}")
print(f"Minimum p-value: {min_p_value:.4f}")

458
2595
Estimated break point date: 1995-05-09 00:00:00
Maximum Wald statistic: 39.4349
Minimum p-value: 0.0000


In [ ]:
fs2 <- Fstats(ds~intj+lag(ds), data = Dat1, from = 0.15 , to = 0.85)
fs2_1 <- fs2$Fstats
Dat4 <- Dat1[(459:2597),] #両端15％をカット
Dat5 <- cbind(Dat4, fs2_1)
Dat5[,1] <- as.Date(Dat5[,1],"%m/%d/%Y")
plot(Dat5$date, Dat5$fs2_1, type="l")
max(Dat5$fs2_1)
Dat5[Dat5$fs2_1 >= 39.4,] #構造変化点


構造変化日は1995/5/10となっているので、TBは1995/5/9

以下では、構造変化前後で別々に推定する

In [22]:
subset_condition_1 = (df['date'] <= '1995-05-09')
Out5 = sm.OLS(df.loc[subset_condition_1, 'ds'], sm.add_constant(df.loc[subset_condition_1, ['intj', 'lag_ds']])).fit()
print(Out5.summary())

                            OLS Regression Results                            
Dep. Variable:                     ds   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     12.76
Date:                Sun, 22 Sep 2024   Prob (F-statistic):           3.35e-06
Time:                        00:19:07   Log-Likelihood:                -1043.4
No. Observations:                1069   AIC:                             2093.
Df Residuals:                    1066   BIC:                             2108.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0271      0.020     -1.340      0.1

In [24]:
subset_condition_2 = (df['date'] > '1995-05-09')
Out6 = sm.OLS(df.loc[subset_condition_2, 'ds'], sm.add_constant(df.loc[subset_condition_2, ['intj', 'lag_ds']])).fit()
print(Out6.summary())

                            OLS Regression Results                            
Dep. Variable:                     ds   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     28.18
Date:                Sun, 22 Sep 2024   Prob (F-statistic):           8.54e-13
Time:                        00:19:14   Log-Likelihood:                -2238.0
No. Observations:                1984   AIC:                             4482.
Df Residuals:                    1981   BIC:                             4499.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0070      0.017      0.415      0.6